In [1]:
import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

### Fountains

In [2]:
fountains = pd.read_csv(os.getenv("fountains"))
fountains.columns = fountains.columns.str.lower()

In [3]:
fountains = gpd.GeoDataFrame(fountains, geometry=gpd.points_from_xy(fountains.longitude, fountains.latitude))

In [4]:
print(fountains.crs)

None


In [5]:
fountains = fountains.set_crs('epsg:4326')

In [6]:
print(fountains.crs)

epsg:4326


In [7]:
columns = ["precision_localisation", "date_installation", "etat", "x", "y"]
fountains = fountains.drop(columns=columns)

In [8]:
fountains.head(1)

,id,arrondissement,nom_parc_lieu,proximité_jeux_repère,intersection,remarque,longitude,latitude,geometry
0,1,Saint-Léonard,Delorme,Bocce,rue Loranger/rue Péguy,NaN,-73.589462,45.592012,POINT (-73.58946 45.59201)


### Urban furniture

In [9]:
mobilier = gpd.read_file(os.getenv("mobilier"))
mobilier.columns = mobilier.columns.str.lower()

In [10]:
print(mobilier.crs)

EPSG:4326


In [11]:
mobilier = mobilier.to_crs("epsg:4326")

In [12]:
print(mobilier.crs)

EPSG:4326


In [13]:
columns = ["element", "inspectdat", "actif", "ancree", "instaldate", "materiau", "grarg",
           "grasec", "indexgparc", "typeancrag", "modele", "x", "y"]
mobilier = mobilier.drop(columns=columns)

In [14]:
mobilier.head(1)

,objectid,elementdes,utilisatio,remarque,nom_parc,long,lat,geometry
0,91,table a pique-nique (TPN),I-Intense,remisage de nombreuses tpn d'événements,PARC LA FONTAINE,-73.569832,45.528918,POINT (-73.56983 45.52892)


### calcul the colsest distance between fountain and urban furniture 

In [15]:
fountains_utm = fountains.to_crs(epsg=32188)
mobilier_utm = mobilier.to_crs(epsg=32188)

In [16]:
print(fountains_utm.crs)
print(mobilier_utm.crs)

EPSG:32188
EPSG:32188


In [17]:
fountain_w_furniture_data = fountains_utm.sjoin_nearest(mobilier_utm, distance_col="nearest_urban")

In [18]:
min(fountain_w_furniture_data["nearest_urban"])

0.0003687759228418947

In [19]:
max(fountain_w_furniture_data["nearest_urban"])

5141.165468421593

In [20]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [21]:
part = fountain_w_furniture_data[fountain_w_furniture_data.nom_parc_lieu.str.contains("Royal", na=False)]
for _, row in part.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [22]:
m

In [23]:
part = fountain_w_furniture_data[fountain_w_furniture_data.nom_parc_lieu.str.contains("Royal", na=False)]

for _, row in part.iterrows():
    lat = row.lat
    lon = row.long
    folium.CircleMarker(
        location=[lat, lon], 
        radius=2,
        color="red",
        fill=True,
        fill_opacity=0.7,
        fill_color="red",
        popup=row["objectid"]).add_to(m)

In [24]:
m